# 오류단어 존재하는 문장의 전체 단어 고려

오류 단어 있는 문장의 모든 명사에 대해 고려해준다.

이렇게하니깐 단어 변경 시 오류가 발생... 확률을 바꿔가며 돌려봐야할듯.

In [1]:
from eunjeon import Mecab
tagger = Mecab()

f=open(r'C:\Users\IBK\Desktop\fluid_engineering.txt',encoding="utf8")

text=[]
while True:
    line=f.readline()
    if tagger.nouns(line)!=[]:
        text.append(tagger.nouns(line))
    if not line:
        break
#print(text)
f.close()

In [2]:
from gensim.models import Word2Vec
model = Word2Vec(text,size=300,window=5,min_count=5,workers=4,sg=1)
# skip-gram 방식으로 학습완료

In [3]:
text_list=sum(text,[])
text_list=set(text_list)
text_list=list(text_list) #중복되는 단어를 제거하고 명사로 이루어진 리스트
#이 리스트에 포함되지 않는 단어는 오류 단어로 간주한다.
#print(text_list)
print(len(text_list))

1880


In [4]:
except_words='흠,예,그게,네,아,어,뭐,음,의,시,은,는,쪽,체,자,이,저,그,때,우리,여러분,번,요,오,앞,뒤,것,여기,저기,이것,저것,이게,저게,이걸,저걸,이번,저번,이런,저런'
excpet_words=except_words.split(',')

In [5]:
# MAIN
import time #실제 자막 나오는 것처럼 한 줄씩 나오는 것처럼 보이기 위해 사용
from eunjeon import Mecab 
from korean_romanizer import *
import jellyfish
from gensim.models import Word2Vec

tagger = Mecab()

def find_error_word(): #오류가 있는지 확인
    error_word=[] #오류단어를 저장
    for i in jamak_nouns:
        if i not in text_list:
            error_word.append(i)
    return error_word

def nearby_error_word(): #오류 단어 앞뒤의 단어 or 앞의 두 단어(오류단어가 가장 마지막에 위치한 경우) 뽑기
    
    if error_word==[]:
        return []
    
    else:
        check_nouns=[]#오류단어 앞뒤의 단어 저장

        for i in jamak_nouns:
            if i not in error_word:
                check_nouns.append(i)
                
    return check_nouns


#----------------------------------------------------오류단어가 바로 옆에 붙어있는 경우에는??

def check_word_list(): #오류 단어 근처의 단어에 대해 연관성 높은 단어의 리스트를 저장
    word_list=[]
    
    if error_word==[]:
        return []

    else:
        for i in range(len(check_nouns)):
            
            for j in check_nouns[i]:
                try:
                    model_result=model.wv.most_similar(j)
                
                except:
                    pass
                    
                for k in model_result:
                    if k[0] not in except_words:
                        word_list.append(k[0])
   
    return word_list


def romanizing(): #word_list의 한글 발음을 로마자로 변환
    pronounce=[] # word_list의 발음을 저장.
    if error_word == []:
        return []
    else:
        for i in range(len(word_list)):
            a=Romanizer(word_list[i])
            pronounce.append(a.romanize())
    return pronounce



def similarity(): #유사도 측정
    probability=[]
    
    if error_word == []:
        return []
    
    else:
        for e in error_word:
            a=Romanizer(e).romanize()
            prob=[]
            
            for j in range(len(pronounce)):
                prob1=jellyfish.jaro_winkler_similarity(a, pronounce[j])
                prob2=jellyfish.jaro_similarity(a, pronounce[j])
                prob3=1-((jellyfish.levenshtein_distance(a, pronounce[j]))/10)
                prob.append(prob3+(prob1+prob2)/2)
            probability.append(prob)
    return probability



def word_change(): #오류단어를 교체
    correct_word=[]
    if error_word==[]:
        return jamak
    else:
        change_word=[]
        for i in range(len(probability)):
            err_word_index=probability[i].index(max(probability[i]))
            correct_word.append(word_list[err_word_index])
            
        for a in range(len(jamak)):
            for b in range(len(error_word)):
                if jamak[a] == error_word[b]:
                    jamak[a]=correct_word[b]
    return jamak



In [6]:

f=open(r'C:\Users\IBK\fluid_lecture.txt',encoding="utf8")


while True:
    line=f.readline()
    jamak_nn=tagger.nouns(line) #자막에 나오는 의미있는 명사를 찾기
    

    #time.sleep(2) # 실제 자막나오듯이 보기 위해 시간 딜레이
    
    if not line:
        break
    
    jamak_nouns=[]
    for w in jamak_nn:
        if w not in except_words:
            jamak_nouns.append(w)
    
    if tagger.morphs(line)!=[]: #자막문장을 품사별로 끊기
        jamak=tagger.morphs(line)
    
    jamak_line=[]
    for i in range(len(jamak)): #자막문장을 품사별로 끊은 것에 순서를 부여
            jamak_line.append([jamak[i],i])
            
    jamak_nouns_line=[]
    for i in range(len(jamak_line)): #자막문장에 나오는 명사가 그 문장에서 가지는 순번
            for j in range(len(jamak_nouns)):
                if jamak_nouns[j] == jamak_line[i][0]:
                    jamak_nouns_line.append(jamak_line[i])
    
    print('자막 = ',jamak)
    #print('자막에 순서를 달은 것 = ',jamak_line)
    
    print('자막에 나오는 명사 = ',jamak_nouns)
    #print('자막의 명사의 순서 = ',jamak_nouns_line)
    
    error_word=find_error_word()
    print('오류 단어 = ',error_word)
    
    check_nouns=nearby_error_word()
    print('오류 단어 근처의 단어 = ',check_nouns)
    
    word_list=check_word_list()
    print('오류 단어에 위치할 확률이 높은 단어 = ',word_list)
    
    pronounce=romanizing()
    print('오류 단어에 위치할 확률이 높은 단어의 발음 = ',pronounce)
   
    probability=similarity()
    print('오류단어와의 발음의 유사성 계산 = ',probability)
    
    change_word=word_change()
    print('최종 변환된 자막 = ',change_word, '\n')
    
    # 자막으로 전환
    line=list(line)
    space_list=[]
    for index, value in enumerate(line):
        if value==' ':
            space_list.append(index)
    
    change_word=list(''.join(change_word))
    for i in space_list:
        change_word.insert(i,' ')
    final_jamak=''.join(change_word)
    print('최종 자막 = ',final_jamak,'\n')
    
f.close()


자막 =  ['자', '여러분', '오늘', '은', '유체', '정역학', '에', '대해', '알아봅시다', '먼저', '정역학', '이란', '무엇', '일까요']
자막에 나오는 명사 =  ['유체', '정역학', '정역학', '무엇']
오류 단어 =  []
오류 단어 근처의 단어 =  []
오류 단어에 위치할 확률이 높은 단어 =  []
오류 단어에 위치할 확률이 높은 단어의 발음 =  []
오류단어와의 발음의 유사성 계산 =  []
최종 변환된 자막 =  ['자', '여러분', '오늘', '은', '유체', '정역학', '에', '대해', '알아봅시다', '먼저', '정역학', '이란', '무엇', '일까요'] 

최종 자막 =  자 여러분 오늘은 유체 정역학에 대해 알아봅시다 먼저 정역학이란 무엇일까요 

자막 =  ['정적', '상태', '에서', '의', '역학', '을', '정역학', '이', '라고', '할', '수', '있', '겠', '지요', '어', '그런데', '균일', '속도', '로', '움직이', '고', '있', '을', '때', '도', '우리', '는', '정적', '상태', '라고', '합니다']
자막에 나오는 명사 =  ['정적', '상태', '역학', '정역학', '수', '균일', '속도', '정적', '상태']
오류 단어 =  []
오류 단어 근처의 단어 =  []
오류 단어에 위치할 확률이 높은 단어 =  []
오류 단어에 위치할 확률이 높은 단어의 발음 =  []
오류단어와의 발음의 유사성 계산 =  []
최종 변환된 자막 =  ['정적', '상태', '에서', '의', '역학', '을', '정역학', '이', '라고', '할', '수', '있', '겠', '지요', '어', '그런데', '균일', '속도', '로', '움직이', '고', '있', '을', '때', '도', '우리', '는', '정적', '상태', '라고', '합니다'] 

최종 자막 =  정적 상태에서의 역학을 정역학이라고 할 수 있겠지요 어